In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Import library-library
import os
import glob
from tqdm import tqdm
import numpy as np

# Data Preparation and Preprocessing
import pandas as pd
import re
from string import digits

# Word Embedding
import joblib
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1')
from keybert import KeyBERT
kw_extractor = KeyBERT('distilbert-base-nli-mean-tokens')
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

# Input and Expansion Query
import nltk
import math
from textblob import TextBlob
from yake import KeywordExtractor
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk import tokenize
from operator import itemgetter
from nltk.tokenize import word_tokenize
from textblob import TextBlob
NLTK_StopWords = stopwords.words('indonesian')
from typing import Union

Downloading:   0%|          | 0.00/795 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/724 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
excluded_words = ["tempat", "waktu"]
NLTK_StopWords.append(["detik", "detikjatim", "detikjateng", "detikjabar", "detiksulsel", "detiksumbar", "detikbali", "detikpapua", "detiksulteng", "detikmaluku", "detjatim", "detikcom"])

In [ ]:
def preprocessing(berita):
    s = berita.lower()
    s = s.replace('\n', ' ')
    s = s.replace('\r', ' ')
    s = s.replace(' o ', ' ')
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    s = re.sub(r'[0-9]', ' ', s)
    tokens = [token for token in s.split(" ") if token != ""]
    T = [t for t in tokens if ((t in excluded_words) or (t not in NLTK_StopWords))]
    return T

In [ ]:
df_total = pd.read_csv('df_total.csv')
df_total = df_total[pd.notnull(df_total['description'])]
print(df_total.info())
print ('-'*90)
document_text= joblib.load('desc_text_train.pkl')
print(len(document_text))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1804 entries, 0 to 1803
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1804 non-null   object
 1   date         1804 non-null   object
 2   description  1804 non-null   object
 3   source       1804 non-null   object
dtypes: object(4)
memory usage: 70.5+ KB
None
------------------------------------------------------------------------------------------
1623


In [ ]:
df_test = pd.read_csv('df_test.csv')
df_test = df_test[pd.notnull(df_test['description'])]
print(df_test.info())
print ('-'*90)
document_text_test= joblib.load('desc_text_test.pkl')
print(len(document_text_test))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181 entries, 0 to 180
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        181 non-null    object
 1   date         181 non-null    object
 2   description  181 non-null    object
 3   source       181 non-null    object
dtypes: object(4)
memory usage: 7.1+ KB
None
------------------------------------------------------------------------------------------
181


In [ ]:
df_train = pd.read_csv('df_train.csv')
df_train = df_train[pd.notnull(df_train['description'])]
print(df_train.info())
print ('-'*90)
document_text_train= joblib.load('desc_text_train.pkl')
print(len(document_text_train))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1623 entries, 0 to 1622
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1623 non-null   object
 1   date         1623 non-null   object
 2   description  1623 non-null   object
 3   source       1623 non-null   object
dtypes: object(4)
memory usage: 63.4+ KB
None
------------------------------------------------------------------------------------------
1623


In [ ]:
bow_kriminalitas_read = pd.read_csv('bow_kriminalitas.csv')
bow_kriminalitas_text = []

for i in range(0, bow_kriminalitas_read.shape[0]):
    bow_kriminalitas_text.append(bow_kriminalitas_read.iloc[i,1])

In [ ]:
bow_where_read = pd.read_csv('bow_where.csv')
bow_where_text = []

for i in range(0, bow_where_read.shape[0]):
    bow_where_text.append(bow_where_read.iloc[i,1])

In [ ]:
bow_when_read = pd.read_csv('bow_when.csv')
bow_when_text = []

for i in range(0, bow_when_read.shape[0]):
    bow_when_text.append(bow_when_read.iloc[i,1])

In [ ]:
bow_who_read = pd.read_csv('bow_who.csv')
bow_who_text = []

for i in range(0, bow_who_read.shape[0]):
    bow_who_text.append(bow_who_read.iloc[i,1])

In [ ]:
# Ambil parent dari bow
bow_list_what = []
bow_list_when = []
bow_list_where = []
bow_list_who = []

for i in range(0, df_bow_what.shape[0]):
  bow_list_what.append(df_bow_what.iloc[i, 1])

for i in range(0, df_bow_when.shape[0]):
  bow_list_when.append(df_bow_when.iloc[i, 1])

for i in range(0, df_bow_where.shape[0]):
  bow_list_where.append(df_bow_where.iloc[i, 1])

for i in range(0, df_bow_who.shape[0]):
  bow_list_who.append(df_bow_who.iloc[i, 1])

print({
  'What': len(bow_list_what),
  'When': len(bow_list_when),
  'Where': len(bow_list_where),
  'Who': len(bow_list_who)
})


{'What': 42622, 'When': 74411, 'Where': 91802, 'Who': 85669}


In [ ]:
def cari_dokpertama(kueriAsli): 
    kueriPre=preprocessing(kueriAsli)
    kueriPre= " ".join (kueriPre)
    hasilSearch=[]
    tfidf_matrix = joblib.load('tfidf_train.pkl')
    tfidf_vectorizer = joblib.load('vectorizer.pkl')
    query_vec= tfidf_vectorizer.transform([kueriPre])
    results=cosine_similarity(tfidf_matrix, query_vec).reshape((-1))
    for i in results.argsort()[-5:][::-1]:
        hasilSearch.append(df_total.iloc[i,-1])
    hasilSearch=". ".join(hasilSearch)
    return hasilSearch

In [ ]:
##Keywords Extraction with YAKE
def keyword_yake(hasilSearch):
    keywordYake=[]

    k_extractor = KeywordExtractor(lan="id", n=1, top=10)
    k_extractor2 = KeywordExtractor(lan="id", n=2, top=10)
    keywords = k_extractor.extract_keywords(text=hasilSearch)
    keywords = k_extractor2.extract_keywords(text=hasilSearch)
    keywordYake = [x for x, y in keywords]
    #keywordYake.append(keywords)
    #print (keywordYake)
    return keywordYake
#print("Keywords of article\n", keywords)

In [ ]:
#Keywords Extraction with TFIDF
def keyword_tfidf(hasilSearch):

    keywordtfidf=[]
    keywordtfidf2=[]

    #doc = 'بَاب فرض الْوضُوء وسننه وهيآته وَفرض الْوضُوء سِتّ خِصَال النِّيَّة عمند غسل الْوَجْه وَغسل الْوَجْه وَغسل الذراعين مَعَ الْمرْفقين وَمسح مَا قل من الرَّأْس وَغسل الرجلَيْن مَعَ الْكَعْبَيْنِ وَالتَّرْتِيب وعَلى قَول الْوَلَاء وسننه عشر خِصَال خمس مِنْهَا قبل غسل الْوَجْه وَهِي التَّسْمِيَة وَغسل الْكَفَّيْنِ والمضمضة وَالِاسْتِنْشَاق وَالْمُبَالغَة فيههما إِلَّا للصَّائِم وَخمْس بعد غسل الْوَجْه وَهِي تَقْدِيم الْيُمْنَى على ليسرى وَمسح جَمِيع الرَّأْس وَمسح الْأُذُنَيْنِ ظاهرهما وباطنهما وَإِدْخَال الأصبعين فيهمَا وتخليل أَصَابِع الرجلَيْن . وَغسل دَاخل الْكَعْبَيْنِ وَلَيْسَ مسح لعنق من سنَنه وفضيلته تكراره ثَلَاثًا وزالواجب فِيهِ مرّة والمرتان أفضل وَالثَّلَاث أكمل وهيآته أَن يبْدَأ فِي تَطْهِير الْأَعْضَاء بمواضع الِابْتِدَاء . فَإِن اقْتصر على فروضه استة أَجزَأَهُ وَإِن ضيع حَظّ نَفسه فِيمَا ترك'
    total_words = re.sub(r'[^\w]', ' ', hasilSearch)
    total_words = total_words.lower().split()
    #print (total_words)
    total_word_length = len(total_words)
    total_sentences = tokenize.sent_tokenize(hasilSearch)
    total_sent_len = len(total_sentences)

    tf_score = {}
    for each_word in total_words:
        #print (each_word)
        each_word = each_word.replace('.','')
        if each_word not in NLTK_StopWords:
            if each_word in tf_score:
                tf_score[each_word] += 1
            else:
                tf_score[each_word] = 1

    # Dividing by total_word_length for each dictionary element
    tf_score.update((x, y/int(total_word_length)) for x, y in tf_score.items())
    #print(tf_score)
    def check_sent(word, sentences): 
        final = [all([w in x for w in word]) for x in sentences] 
        sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
        return int(len(sent_len))

    idf_score = {}
    for each_word in total_words:
        #print (each_word)
        each_word = each_word.replace('.','')
        if each_word not in NLTK_StopWords:
            if each_word in idf_score:
                idf_score[each_word] = check_sent(each_word, total_sentences)
            else:
                idf_score[each_word] = 1

    # Performing a log and divide
    idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())

    #print(idf_score)
    tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}
    #print(tf_idf_score)
    def get_top_n(dict_elem, n):
        result = dict(sorted(dict_elem.items(), key = itemgetter(1), reverse = True)[:n]) 
        hasil =list(result.keys())
        #print(list(result.keys()))        
        return hasil
    #print(get_top_n(tf_idf_score, 25))
    #print(len(get_top_n(tf_idf_score, 1)))
    keywordtfidf.append(get_top_n(tf_idf_score, 25))
    for i in range(len(keywordtfidf)):
        #print (i)
        totalKw=0
        totalKw=len(keywordtfidf[i])
        for j in range(totalKw):
            #print (j)
            keywordtfidf2.append(keywordtfidf[i][j])
    #print (keywordtfidf2)
    return keywordtfidf2

In [ ]:
#Keywords Extraction with BERT
def keyword_bert(hasilSearch):

    keywordbert=[]

    #for j in range(len(array_text)):
    keyword1 = kw_extractor.extract_keywords(hasilSearch, top_n=10, keyphrase_ngram_range=(1, 1))
    keyword2 = kw_extractor.extract_keywords(hasilSearch, top_n=10, keyphrase_ngram_range=(1, 2))

    #print("Keywords of article\n", keywords)
    for i in range (0,len (keyword1)):
        keywordbert.append(keyword1[i][0])
        keywordbert.append(keyword2[i][0])
    #print (keywordbert)
    return keywordbert

In [ ]:
def rangking (keywordGabung,kueriAsli):
    kandidatFinalCek=[]
    kandidatFinalFix=[]
    for i in keywordGabung:
        if (i not in kandidatFinalCek and i!=0):
            kandidatFinalCek.append(i)
    queries=[kueriAsli]
    query_embeddings = embedder.encode(queries)
    corpus_embeddings4 = embedder.encode(kandidatFinalCek)
    # Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
    closest_n = 30
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings4, 'cosine')[0]
        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
        for idx, distance in results[0:closest_n]:
            kandidatFinalFix.append(kandidatFinalCek[idx])
    print ('kandidatFinalFix: ', kandidatFinalFix)
    return kandidatFinalFix

In [ ]:
def keyword_BOW(keywordBOW, kueriAsli):
    cekDuplicate = []
    kandidatFix = []

    for i in keywordBOW:
        if(i not in cekDuplicate and i!=0):
            cekDuplicate.append(i)

    queries=[kueriAsli]
    query_embeddings = embedder.encode(queries)
    corpus_embeddings4 = embedder.encode(cekDuplicate)
    
    # Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
    closest_n = 10
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings4, 'cosine')[0]
        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
        for idx, distance in results[0:closest_n]:
            kandidatFix.append(cekDuplicate[idx])
    return kandidatFix

In [ ]:
def kandidatFix(kueriAsli,bow):
    
    kueri=preprocessing(kueriAsli)
    kueri= [" ".join (kueri)]
    print (kueri)
    hasilkandidat=[]
    keywordGabung=[]
    kandidatFix=[]
    kueriFix=[]

    hasilSearch=cari_dokpertama(kueriAsli)
    keywordYake=keyword_yake(hasilSearch)
    keywordtfidf2=keyword_tfidf(hasilSearch)
    keywordbert=keyword_bert (hasilSearch)
    keywordBOW=keyword_BOW(bow, kueriAsli)

    for i in keywordYake:
        keywordGabung.append(i)
    for i in keywordtfidf2:
        keywordGabung.append(i)
    for i in keywordbert:
        keywordGabung.append(i)
    for i in keywordBOW:
        keywordGabung.append(i)
    hasilrank=rangking(keywordGabung,kueriAsli)
    # print(hasilrank)
    for i in hasilrank:
        kueriFix.append(i)
    for j in kueriFix:
        hasilkandidat.append(j)
    kueriFix=[preprocessing(i) for i in kueriFix]
    for i in kueriFix:
        for j in i:
            kandidatFix.append(j)
    kandidatFix= [" ".join (kandidatFix)]
    print ('*'*120)

    return(kandidatFix,keywordGabung,keywordBOW,hasilrank)

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
hasilDokumenWhat=[]
hasilDokumenWho=[]
hasilDokumenWhen=[]
hasilDokumenWhere=[]

kueriAsliWhat='apa sebenarnya kejadian yang terjadi diberita tersebut'
kueriAsliWhere='di daerah mana kejadian itu terjadi'
kueriAsliWho='siapa pelaku kejadian ini'
kueriAsliWhen='Kapan waktu kejadian tersebut'

kandidatFixWhat,keywordGabungWhat,keywordBOW_What, hasilrankWhat = kandidatFix(kueriAsliWhat, bow_kriminalitas_text)
kandidatFixWhere,keywordGabungWhere,keywordBOW_Where, hasilrankWhere = kandidatFix(kueriAsliWhere, bow_where_text)
kandidatFixWho,keywordGabungWho,keywordBOW_Who, hasilrankWho = kandidatFix(kueriAsliWho, bow_who_text)
kandidatFixWhen,keywordGabungWhen,keywordBOW_When, hasilrankWhen = kandidatFix(kueriAsliWhen, bow_when_text)


['kejadian diberita']
kandidatFinalFix:  ['terjadinya', 'mempedulikan', 'dipersalahkan', 'pelakunya', 'merebaknya', 'sepengetahuan', 'ntoutoume', 'kepentingannya', 'niatnya', 'dijatuhkannya', 'detik com', 'detik', 'com', 'com www', 'www', 'www detik']
************************************************************************************************************************
['daerah kejadian']
kandidatFinalFix:  ['tempatnya', 'disekitarnya', 'berlokasi', 'ditempat', 'dimana', 'lokasinya', 'ternyata', 'dilokasi', 'дэлхийн', 'disebelahnya', 'detik com', 'detik', 'www', 'com www', 'com', 'www detik']
************************************************************************************************************************
['pelaku kejadian']
kandidatFinalFix:  ['pelakunya', 'disalahkan', 'dipersalahkan', 'kesalahannya', 'mempersalahkan', 'menyalahkannya', 'pembuatnya', 'konspirator', 'diperbuatnya', 'berlakunya', 'detik com', 'detik', 'com', 'www', 'com www', 'www detik']
***********************

In [ ]:
j=1

testing_data = []
for i in range(0, len(document_text)-1):
    hasilWhat=[]
    hasilWho=[]
    hasilWhere=[]
    teks=df_total.iloc[i,-2]
    tfidf_vectorizer = joblib.load('vectorizer.pkl')
    # tfidf_matrix = joblib.load('tfidf_test.pkl')
    tfidf_matrix = tfidf_vectorizer.fit_transform([teks])

    query_vec_What= tfidf_vectorizer.transform(kandidatFixWhat)
    results_what=cosine_similarity(tfidf_matrix, query_vec_What).reshape((-1))
    hasilDokumenWhat.append(df_total.iloc[i,2])
    for a in hasilrankWhat:
        cariW = re.findall(a,hasilDokumenWhat[i])
        #print(cariW)
        if cariW:
            hasilWhat.append(a)
    
    data = [i,df_total.iloc[i,2],'what',kueriAsliWhat, hasilWhat, results_what]

    testing_data.append(data)

    query_vec_Where= tfidf_vectorizer.transform(kandidatFixWhere)
    results_where=cosine_similarity(tfidf_matrix, query_vec_Where).reshape((-1))
    hasilDokumenWhere.append(df_total.iloc[i,2])
    for a in hasilrankWhere:
        cariW = re.findall(a,hasilDokumenWhere[i])
        #print(cariW)
        if cariW:
            hasilWhere.append(a)
    
    data = [i,df_total.iloc[i,2],'where',kueriAsliWhere, hasilWhere, results_where]

    testing_data.append(data)

    query_vec_Who= tfidf_vectorizer.transform(kandidatFixWho)
    results_who=cosine_similarity(tfidf_matrix, query_vec_Who).reshape((-1))
    hasilDokumenWho.append(df_total.iloc[i,2])
    for a in hasilrankWho:
        cariW = re.findall(a,hasilDokumenWho[i])
        #print(cariW)
        if cariW:
            hasilWho.append(a)
    
    data = [i,df_total.iloc[i,2],'who',kueriAsliWho, hasilWho, results_who]

    testing_data.append(data)
    
    # print (j)
    # print("No ID Dokumen  : ", i)
    # print("Tanggal        : ", df_test.iloc[i,1])
    # print("Isi berita     : ", df_test.iloc[i,2])
    # print ("Hasil W       : ",hasilW)
    # print("(Kemiripan: %.4f) " % results)
    # print ('*'*120)
    j+=1

In [ ]:
#write to csv
writer = pd.DataFrame(testing_data, columns=['No Document','Description', 'W', 'Kueri W','hasilW', 'Kemiripan'])
writer.to_csv('QE_Stat_result.csv', index=False, sep=',')

In [ ]:
QE_result_read = pd.read_csv('QE_Stat_result.csv')
QE_result_read.head()

,No Document,Description,W,Kueri W,hasilW,Kemiripan
0,0,penindakan peredaran kapolsek palmerah akp dod...,what,apa sebenarnya kejadian yang terjadi diberita ...,[],[0.]
1,0,penindakan peredaran kapolsek palmerah akp dod...,where,di daerah mana kejadian itu terjadi,[],[0.]
2,0,penindakan peredaran kapolsek palmerah akp dod...,who,siapa pelaku kejadian ini,[],[0.]
3,1,tanjung priok jakarta utara digerebek ratusan ...,what,apa sebenarnya kejadian yang terjadi diberita ...,[],[0.]
4,1,tanjung priok jakarta utara digerebek ratusan ...,where,di daerah mana kejadian itu terjadi,[],[0.]


In [ ]:
# #kueri what
# kueriAsliWhat='Apa kejadian yang terjadi pada berita tersebut'
# kueriWhat=preprocessing(kueriAsliWhat)
# kueriWhat= [" ".join (kueriWhat)]
# print (kueriWhat)
# hasilkandidatWhat=[]
# keywordGabungWhat=[]
# kandidatFixWhat=[]
# kueriFixWhat=[]
# hasilDokumenWhat=[]

# hasilSearchWhat=cari_dokpertama(kueriAsliWhat)
# keywordYakeWhat=keyword_yake(hasilSearchWhat)
# keywordtfidf2What=keyword_tfidf(hasilSearchWhat)
# keywordbertWhat=keyword_bert (hasilSearchWhat)
# keywordBOW_What=keyword_BOW(bow_kriminalitas_text, kueriAsliWhat)

# for i in keywordYakeWhat:
#     keywordGabungWhat.append(i)
# for i in keywordtfidf2What:
#     keywordGabungWhat.append(i)
# for i in keywordbertWhat:
#     keywordGabungWhat.append(i)
# for i in keywordBOW_What:
#     keywordGabungWhat.append(i)
# hasilrankWhat=rangking(keywordGabungWhat,kueriAsliWhat)
# # print(hasilrank)
# for i in hasilrankWhat:
#     kueriFixWhat.append(i)
# for j in kueriFixWhat:
#     hasilkandidatWhat.append(j)
# kueriFixWhat=[preprocessing(i) for i in kueriFixWhat]
# for i in kueriFixWhat:
#     for j in i:
#         kandidatFixWhat.append(j)
# kandidatFixWhat= [" ".join (kandidatFixWhat)]
# print ('*'*120)


['kejadian berita']


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


kandidatFinalFix:  ['terjadinya', 'mempedulikan', 'kepentingannya', 'pengungkapan', 'investigasi', 'dipersalahkan', 'merebaknya', 'timbulnya', 'pelakunya', 'dedi', 'detik com', 'detik', 'com', 'www', 'com www', 'www detik']
************************************************************************************************************************


In [ ]:
# #kueri where
# kueriAsliWhere='Dimana peristiwa itu terjadi'
# kueriWhere=preprocessing(kueriAsliWhere)
# kueriWhere= [" ".join (kueriWhere)]
# print (kueriWhere)
# hasilkandidatWhere=[]
# keywordGabungWhere=[]
# kandidatFixWhere=[]
# kueriFixWhere=[]
# hasilDokumenWhere=[]

# hasilSearchWhere=cari_dokpertama(kueriAsliWhere)
# keywordYakeWhere=keyword_yake(hasilSearchWhere)
# keywordtfidf2Where=keyword_tfidf(hasilSearchWhere)
# keywordbertWhere=keyword_bert (hasilSearchWhere)
# keywordBOW_Where=keyword_BOW(bow_where_text, kueriAsliWhere)

# for i in keywordYakeWhere:
#     keywordGabungWhere.append(i)
# for i in keywordtfidf2Where:
#     keywordGabungWhere.append(i)
# for i in keywordbertWhere:
#     keywordGabungWhere.append(i)
# for i in keywordBOW_Where:
#     keywordGabungWhere.append(i)
# hasilrankWhere=rangking(keywordGabungWhere,kueriAsliWhere)
# # print(hasilrank)
# for i in hasilrankWhere:
#     kueriFixWhere.append(i)
# for j in kueriFixWhere:
#     hasilkandidatWhere.append(j)
# kueriFixWhere=[preprocessing(i) for i in kueriFixWhere]
# for i in kueriFixWhere:
#     for j in i:
#         kandidatFixWhere.append(j)
# kandidatFixWhere= [" ".join (kandidatFixWhere)]
# print ('*'*120)

['dimana peristiwa']


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


kandidatFinalFix:  ['tempatnya', 'dimana', 'kemana', 'lokasinya', 'ditempat', 'ternyata', 'dilokasi', 'tempat', 'berlokasi', 'ketika', 'detik com', 'detik', 'www', 'com www', 'com', 'www detik']
************************************************************************************************************************


In [ ]:
# #kueri when
# kueriAsliWhen='Kapan peristiwa itu terjadi'
# kueriWhen=preprocessing(kueriAsliWhen)
# kueriWhen= [" ".join (kueriWhen)]
# print (kueriWhen)
# hasilkandidatWhen=[]
# keywordGabungWhen=[]
# kandidatFixWhen=[]
# kueriFixWhen=[]
# hasilDokumenWhen=[]

# hasilSearchWhen=cari_dokpertama(kueriAsliWhen)
# keywordYakeWhen=keyword_yake(hasilSearchWhen)
# keywordtfidf2When=keyword_tfidf(hasilSearchWhen)
# keywordbertWhen=keyword_bert (hasilSearchWhen)
# keywordBOW_When=keyword_BOW(bow_when_text, kueriAsliWhen)

# for i in keywordYakeWhen:
#     keywordGabungWhen.append(i)
# for i in keywordtfidf2When:
#     keywordGabungWhen.append(i)
# for i in keywordbertWhen:
#     keywordGabungWhen.append(i)
# for i in keywordBOW_When:
#     keywordGabungWhen.append(i)
# hasilrankWhere=rangking(keywordGabungWhen,kueriAsliWhen)
# # print(hasilrank)
# for i in hasilrankWhere:
#     kueriFixWhen.append(i)
# for j in kueriFixWhen:
#     hasilkandidatWhen.append(j)
# kueriFixWhen=[preprocessing(i) for i in kueriFixWhen]
# for i in kueriFixWhen:
#     for j in i:
#         kandidatFixWhen.append(j)
# kandidatFixWhen= [" ".join (kandidatFixWhen)]
# print ('*'*120)

['peristiwa']


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


kandidatFinalFix:  ['kapankah', 'apakah', 'kenapa', 'kapan', 'mengapakah', 'gimana', 'bagaimana', 'berlakunya', 'mengapa', 'sebabnya', 'detik com', 'detik', 'www', 'www detik', 'com', 'com www']
************************************************************************************************************************


In [ ]:
# #kueri who
# kueriAsliWho='Siapa pelaku dari peristiwa itu'
# kueriWho=preprocessing(kueriAsliWho)
# kueriWho= [" ".join (kueriWho)]
# print (kueriWho)
# hasilkandidatWho=[]
# keywordGabungWho=[]
# kandidatFixWho=[]
# kueriFixWho=[]
# hasilDokumenWho=[]

# hasilSearchWho=cari_dokpertama(kueriAsliWho)
# keywordYakeWho=keyword_yake(hasilSearchWho)
# keywordtfidf2Who=keyword_tfidf(hasilSearchWho)
# keywordbertWho=keyword_bert (hasilSearchWho)
# keywordBOW_Who=keyword_BOW(bow_who_text, kueriAsliWho)

# for i in keywordYakeWho:
#     keywordGabungWho.append(i)
# for i in keywordtfidf2Who:
#     keywordGabungWho.append(i)
# for i in keywordbertWho:
#     keywordGabungWho.append(i)
# for i in keywordBOW_Who:
#     keywordGabungWho.append(i)
# hasilrankWhere=rangking(keywordGabungWho,kueriAsliWho)
# # print(hasilrank)
# for i in hasilrankWhere:
#     kueriFixWho.append(i)
# for j in kueriFixWho:
#     hasilkandidatWho.append(j)
# kueriFixWho=[preprocessing(i) for i in kueriFixWho]
# for i in kueriFixWho:
#     for j in i:
#         kandidatFixWho.append(j)
# kandidatFixWho= [" ".join (kandidatFixWho)]
# print ('*'*120)

In [ ]:
# kueriAsli='pelaku'
# kueri=preprocessing(kueriAsli)
# kueri= [" ".join (kueri)]
# print (kueri)
# hasilkandidat=[]
# keywordGabung=[]
# kandidatFix=[]
# kueriFix=[]
# hasilDokumen=[]

# hasilSearch=cari_dokpertama(kueriAsli)
# keywordYake=keyword_yake(hasilSearch)
# keywordtfidf2=keyword_tfidf(hasilSearch)
# keywordbert=keyword_bert (hasilSearch)
# for i in keywordYake:
#     keywordGabung.append(i)
# for i in keywordtfidf2:
#     keywordGabung.append(i)
# for i in keywordbert:
#     keywordGabung.append(i)
# hasilrank=rangking(keywordGabung,kueriAsli)
# for i in hasilrank:
#     kueriFix.append(i)
# for j in kueriFix:
#     hasilkandidat.append(j)
# kueriFix=[preprocessing(i) for i in kueriFix]
# for i in kueriFix:
#     for j in i:
#         kandidatFix.append(j)
# kandidatFix= [" ".join (kandidatFix)]
# print ('*'*120)
# j=1
# for i in range(0, 5):
#     hasilW=[]
#     teks=df_total.iloc[i,-1]
#     tfidf_vectorizer = TfidfVectorizer()
#     tfidf_matrix = tfidf_vectorizer.fit_transform([teks])
#     query_vec= tfidf_vectorizer.transform(kandidatFix)
#     results=cosine_similarity(tfidf_matrix, query_vec).reshape((-1))
#     hasilDokumen.append(df_total.iloc[i,2])
#     for a in hasilrank:
#         cariW = re.findall(a,hasilDokumen[i])
#         #print(cariW)
#         if cariW:
#             hasilW.append(a)
#     print (j)
#     print("No ID Dokumen  : ", i)
#     print("Tanggal        : ", df_total.iloc[i,1])
#     print("Isi berita     : ", df_total.iloc[i,2])
#     print ("Hasil W       : ",hasilW)
#     print("(Kemiripan: %.4f) " % results)
#     print ('*'*120)
#     j+=1

In [ ]:
# kueriAsli='pencurian'
# kueri=preprocessing(kueriAsli)
# kueri= [" ".join (kueri)]
# print (kueri)
# hasilkandidat=[]
# keywordGabung=[]
# kandidatFix=[]
# kueriFix=[]
# hasilDokumen=[]
# hasilSearch=cari_dokpertama(kueriAsli)
# keywordYake=keyword_yake(hasilSearch)
# keywordtfidf2=keyword_tfidf(hasilSearch)
# keywordbert=keyword_bert (hasilSearch)
# for i in keywordYake:
#     keywordGabung.append(i)
# for i in keywordtfidf2:
#     keywordGabung.append(i)
# for i in keywordbert:
#     keywordGabung.append(i)
# hasilrank=rangking(keywordGabung,kueriAsli)
# for i in hasilrank:
#     kueriFix.append(i)
# for j in kueriFix:
#     hasilkandidat.append(j)
# kueriFix=[preprocessing(i) for i in kueriFix]
# for i in kueriFix:
#     for j in i:
#         kandidatFix.append(j)
# kandidatFix= [" ".join (kandidatFix)]
# print ('*'*120)
# tfidf_matrix =joblib.load( "tfidf.pkl" )
# tfidf_vectorizer = joblib.load( "vectorizer.pkl" ) 
# query_vec= tfidf_vectorizer.transform(kandidatFix)
# results=cosine_similarity(tfidf_matrix, query_vec).reshape((-1))
# j=1
# for i in results.argsort()[-10:][::-1]:
#     print (j)
#     print("No ID Dokumen  : ", i)
#     print("Tanggal        : ", df_total.iloc[i,1])
#     print("Isi berita     : ", df_total.iloc[i,2])
#     print("(Kemiripan: %.4f) " % results[i])
#     hasilDokumen.append(df_total.iloc[i,2])
#     print ('*'*120)
#     j+=1